In [1]:
import pandas as pd
import pathlib, os
import torch
from clai.tooling import tool
from clai.tooling import io
from clai.processing import processor

In [2]:
from clai.modeling.module.model import model

In [3]:
label_list = [
        '8800 СберМегаМаркет. Купить',
        '8800 СберМегаМаркет. Статус доставки и заказа',
        '8800 СберМегаМаркет. Другие вопросы',
        'Соединить с оператором',
        '8800 СберМегаМаркет. Не пришел заказ',
        '8800 СберМегаМаркет. Разводящий вопрос',
        'Да',
        'Прочее',
        '8800 СберМегаМаркет. Качество, комплектация, состав заказа',
        '8800 СберМегаМаркет. Программа лояльности',
        '8800 СберМегаМаркет. Отменить заказ',
        '8800 СберМегаМаркет. Узнать условия',
        '8800 СберМегаМаркет. Обращение',
        '8800 СберМегаМаркет. Изменить заказ',
        '8800 СберМегаМаркет. Изменить личные данные',
        '8800 СберМегаМаркет. Оплата',
        '8800 СберМегаМаркет. Изменить доставку',
        '8800 СберМегаМаркет. Заказ отменили',
        '8800 СберМегаМаркет. Вернуть деньги',
        '8800 СберМегаМаркет. Вернуть товар',
        'Нет',
        '8800 СберМегаМаркет. Получение заказа',
        '8800 СберМегаМаркет. Юридические лица',
        '8800 СберМегаМаркет. Промокод'
    ]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cpu


In [5]:
lm = model.LanguageModel.load("coi")

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tokenizer = io.load_tokenizer(pretrained_model_name_or_path="cointegrated/rubert-tiny")

In [7]:
tokenizer

PreTrainedTokenizerFast(name_or_path='cointegrated/rubert-tiny', vocab_size=29564, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [8]:
data_dir = pathlib.Path.home() / "Dataset" / "sber"
max_seq_len = 128
text_column_name = "input_text"
label_column_name = "topic"
metric = "f1_macro"
train_filename = "train.csv"
test_filename = "test.csv"

In [9]:
processor = processor.Processor.load(
    "klass",
    max_seq_len=max_seq_len,
    tokenizer=tokenizer, 
    data_dir=data_dir, 
    train_filename=train_filename,
    test_filename=test_filename,
    dev_split=0.0,
    label_list=label_list,
    text_column_name=text_column_name, 
    label_column_name=label_column_name,
    metric=metric
)

In [10]:
from clai.flowing import flow

In [11]:
data_silo = flow.Flow.load("haski", processor=processor, batch_size=1)

Preprocessing Dataset /Users/20107004/Dataset/sber/train.csv:   0%|          | 0/19334 [00:00<?, ? Dicts/s]

Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for re

Preprocessing Dataset /Users/20107004/Dataset/sber/test.csv:   0%|          | 0/4784 [00:00<?, ? Dicts/s]

Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for returning problematic ids
Currently no support in Processor for re

In [12]:
from clai.modeling.module.head import klass

In [13]:
prediction_head = klass.TextClassificationHead(num_labels=len(label_list))